In [77]:
# Identify athletes meeting OCTC selection rules

#1. At least 1 athlete per event per gender
#2. Except for 100m, 400m, cap at 3 athletes per event. 100m, 400m capped at 6
#3. Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)
#4. Where althlete qualified in 2 events, to choose the better performing one
#5. For athletes looking to do full time, to write in to SAA for special consideration
#6. Exclude SPEX carded athletes
#7. Except for marathon, age threshold cut off of 40 yrs old for top athlete
#8. No double tapping of prog - potential names in red


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [79]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DOB, COUNTRY, CATEGORY_EVENT, GENDER, WIND, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE DIVISION!='OPEN' AND DIVISION!='Open' AND RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [80]:
athletes.to_csv('athletes.csv', sep=',', encoding='utf-8-sig', index=False)

In [81]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,WIND,COMPETITION,DATE
0,REUBEN SEET HUIJIE,19:04.4,None,11.0,A,5000m,None,None,Long,Male,None,NSG,2023
1,PRANAV SHREEDHAR,18:58.1,None,5.0,A,5000m,None,None,Long,Male,None,NSG,2023
2,JACOB TAN,18:58.2,None,6.0,A,5000m,None,None,Long,Male,None,NSG,2023
3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,A,5000m,None,None,Long,Male,None,NSG,2023
4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,A,5000m,None,None,Long,Male,None,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21868,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,B,4 X 100m relay,None,None,Relay,Female,None,NSG,2022
21869,HWA CHONG INSTITUTION,00:50.1,None,1,A,4 X 100m relay,None,None,Relay,Female,None,NSG,2022
21870,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,C,4 X 400m relay,None,None,Relay,Female,None,NSG,2022
21871,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,B,4 X 400m relay,None,None,Relay,Female,None,NSG,2022


In [82]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'400m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'1500m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'10000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'


# Hurdles

mask = athletes['EVENT'].str.contains(r'80m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'80m hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'80 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100m hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'110m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200m hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'400m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Shot', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Hammer', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'
mask = athletes['EVENT'].str.contains(r'Discus', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'

# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m steeplechase'


# Walk

mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m race walk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m race walk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m race walk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m race walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'




In [83]:
athletes['MAPPED_AGE']=athletes['AGE']

# Map NSG divisions to age bracket. A=17-20, B=15-17, C=13-14

mask = athletes['DIVISION'].str.contains(r'A', na=True)
athletes.loc[mask, 'MAPPED_AGE'] = '18.5'
mask = athletes['DIVISION'].str.contains(r'B', na=True)
athletes.loc[mask, 'MAPPED_AGE'] = '16'
mask = athletes['DIVISION'].str.contains(r'C', na=True)
athletes.loc[mask, 'MAPPED_AGE'] = '13.5'





In [84]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,WIND,COMPETITION,DATE,MAPPED_EVENT,MAPPED_AGE
0,REUBEN SEET HUIJIE,19:04.4,None,11.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
1,PRANAV SHREEDHAR,18:58.1,None,5.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
2,JACOB TAN,18:58.2,None,6.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21868,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,B,4 X 100m relay,None,None,Relay,Female,None,NSG,2022,4 x 100m relay,16
21869,HWA CHONG INSTITUTION,00:50.1,None,1,A,4 X 100m relay,None,None,Relay,Female,None,NSG,2022,4 x 100m relay,18.5
21870,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,C,4 X 400m relay,None,None,Relay,Female,None,NSG,2022,4 x 400m relay,13.5
21871,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,B,4 X 400m relay,None,None,Relay,Female,None,NSG,2022,4 x 400m relay,16


In [85]:
athletes.to_csv('athletes2_relay.csv', sep=',', encoding='utf-8-sig', index=False)


In [86]:
credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND RANK='3'
"""

benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [87]:
benchmarks

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
0,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final
1,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final
2,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final
3,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final
4,Robi Syianturi,14:43.45,3,5000m,Long,Male,SEA Games,Final
5,Tan Htike Soe,31:25.55,3,10000m,Sprint,Male,SEA Games,Final
6,John Cabang,13.855,3,110m hurdles,Hurdles,Male,SEA Games,Final
7,Calvin Quek,50.75,3,400m hurdles,Hurdles,Male,SEA Games,Final
8,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final
9,"Jonathan Nyepa, Khairul Hafiz Jantan, Mohama...",39.36,3,4 x 100m relay,Relay,Male,SEA Games,Final


In [88]:
# rename RESULT column name

benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [89]:
# drop columns

benchmarks.drop(['NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [90]:
benchmarks

,BENCHMARK,EVENT,GENDER
0,10.443,100m,Male
1,21.02,200m,Male
2,46.63,400m,Male
3,1:53.86,800m,Male
4,14:43.45,5000m,Male
5,31:25.55,10000m,Male
6,13.855,110m hurdles,Male
7,50.75,400m hurdles,Male
8,8:55.05,3000m steeplechase,Male
9,39.36,4 x 100m relay,Male


In [91]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()    
    
    if any(s in string for s in l)==True:
        
        if 'm' in metric:
            
            metric=metric.replace('m', '')
            output=float(str(metric))
            
        elif 'GR' in metric:
            
            metric=metric.replace('GR', '')
            output=float(str(metric))


        else:
    
            output=float(str(metric))
        
    else:
        
        searchstring = ":"
        searchstring2 = "."
        substring=str(metric)
        count = substring.count(searchstring)
        count2 = substring.count(searchstring2)
            
        if count==0:
            output=float(substring)
                        
             
        elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                                                
            time=str(metric)
            h, m ,s = time.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
        elif (count==1 and count2==1):
            
            m,s = metric.split(':')
            output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
            
            if output==229.90:
                print(metric, m, s, output, 'here')

                     
        elif (count==1 and count2==2):
            
            metric = metric.replace(".", ":", 1)
            
            h,m,s = metric.split(':')            
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
        elif (count==2 and count2==0):
            
            h,m,s = metric.split(':')
            output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                
    return output

In [92]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,1]
    
    metric=benchmarks.iloc[rowIndex,0]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

In [93]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric
0,10.443,100m,Male,10.443
1,21.02,200m,Male,21.020
2,46.63,400m,Male,46.630
3,1:53.86,800m,Male,113.860
4,14:43.45,5000m,Male,883.450
5,31:25.55,10000m,Male,1885.550
6,13.855,110m hurdles,Male,13.855
7,50.75,400m hurdles,Male,50.750
8,8:55.05,3000m steeplechase,Male,535.050
9,39.36,4 x 100m relay,Male,39.360


In [94]:
# Calculate performance windows
# Jump, throw, pole vault and shotput events are +ve distance, whereas the others are -ve timing

mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put', na=True)

benchmarks.loc[mask, '2pc']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '35pc']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5pc']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2pc']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '35pc']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5pc']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [95]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT']

In [96]:
benchmarks

,BENCHMARK,EVENT,GENDER,Metric,2pc,35pc,5pc,MAPPED_EVENT
0,10.443,100m,Male,10.443,10.65186,10.808505,10.96515,100m
1,21.02,200m,Male,21.020,21.44040,21.755700,22.07100,200m
2,46.63,400m,Male,46.630,47.56260,48.262050,48.96150,400m
3,1:53.86,800m,Male,113.860,116.13720,117.845100,119.55300,800m
4,14:43.45,5000m,Male,883.450,901.11900,914.370750,927.62250,5000m
5,31:25.55,10000m,Male,1885.550,1923.26100,1951.544250,1979.82750,10000m
6,13.855,110m hurdles,Male,13.855,14.13210,14.339925,14.54775,110m hurdles
7,50.75,400m hurdles,Male,50.750,51.76500,52.526250,53.28750,400m hurdles
8,8:55.05,3000m steeplechase,Male,535.050,545.75100,553.776750,561.80250,3000m steeplechase
9,39.36,4 x 100m relay,Male,39.360,40.14720,40.737600,41.32800,4 x 100m relay


In [97]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT,DOB,COUNTRY,CATEGORY_EVENT,GENDER,WIND,COMPETITION,DATE,MAPPED_EVENT,MAPPED_AGE
0,REUBEN SEET HUIJIE,19:04.4,None,11.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
1,PRANAV SHREEDHAR,18:58.1,None,5.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
2,JACOB TAN,18:58.2,None,6.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,A,5000m,None,None,Long,Male,None,NSG,2023,5000m,18.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21868,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,B,4 X 100m relay,None,None,Relay,Female,None,NSG,2022,4 x 100m relay,16
21869,HWA CHONG INSTITUTION,00:50.1,None,1,A,4 X 100m relay,None,None,Relay,Female,None,NSG,2022,4 x 100m relay,18.5
21870,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,C,4 X 400m relay,None,None,Relay,Female,None,NSG,2022,4 x 400m relay,13.5
21871,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,B,4 X 400m relay,None,None,Relay,Female,None,NSG,2022,4 x 400m relay,16


In [98]:
# Left join benchmarks table to athletes table

df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [99]:
df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,DATE,MAPPED_EVENT,MAPPED_AGE,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
0,0,REUBEN SEET HUIJIE,19:04.4,None,11.0,A,5000m,None,None,Long,...,2023,5000m,18.5,4.0,14:43.45,5000m,883.45,901.1190,914.37075,927.6225
1,1,PRANAV SHREEDHAR,18:58.1,None,5.0,A,5000m,None,None,Long,...,2023,5000m,18.5,4.0,14:43.45,5000m,883.45,901.1190,914.37075,927.6225
2,2,JACOB TAN,18:58.2,None,6.0,A,5000m,None,None,Long,...,2023,5000m,18.5,4.0,14:43.45,5000m,883.45,901.1190,914.37075,927.6225
3,3,SAMUEL TOH JUN XIAN,19:03.8,None,8.0,A,5000m,None,None,Long,...,2023,5000m,18.5,4.0,14:43.45,5000m,883.45,901.1190,914.37075,927.6225
4,4,LIEU YEA JAY ERWIN,19:04.1,None,9.0,A,5000m,None,None,Long,...,2023,5000m,18.5,4.0,14:43.45,5000m,883.45,901.1190,914.37075,927.6225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21868,21868,CEDAR GIRLS SECONDARY SCHOOL,00:50.1,None,1,B,4 X 100m relay,None,None,Relay,...,2022,4 x 100m relay,16,31.0,44.58,4 x 100m relay,44.58,45.4716,46.14030,46.8090
21869,21869,HWA CHONG INSTITUTION,00:50.1,None,1,A,4 X 100m relay,None,None,Relay,...,2022,4 x 100m relay,18.5,31.0,44.58,4 x 100m relay,44.58,45.4716,46.14030,46.8090
21870,21870,CEDAR GIRLS SECONDARY SCHOOL,04:26.2,None,1,C,4 X 400m relay,None,None,Relay,...,2022,4 x 400m relay,13.5,32.0,3:39.29,4 x 400m relay,219.29,223.6758,226.96515,230.2545
21871,21871,SINGAPORE SPORTS SCHOOL,04:11.7,None,1,B,4 X 400m relay,None,None,Relay,...,2022,4 x 400m relay,16,32.0,3:39.29,4 x 400m relay,219.29,223.6758,226.96515,230.2545


In [100]:
# Choose 100m,200m and 400m event

relay_df = df.loc[(df['MAPPED_EVENT'] == '100m') | (df['MAPPED_EVENT'] == '200m') | (df['MAPPED_EVENT']=='400m')]

In [101]:
relay_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,DATE,MAPPED_EVENT,MAPPED_AGE,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
65,65,CHUA CHEE HON,00:20.5,None,4.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
66,66,O'NEAL TERENCE,00:33.3,None,5.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
67,67,MARCUS ANG GEE HONG,00:15.7,None,2.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
68,68,DHYAN JEEVANTH,00:17.4,None,3.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
69,69,MOHAMMAD ZULKHAIRI LUTFI BIN MOHD AZAM,00:13.9,None,1.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21829,21829,ELIZABETH LEE SHYIN,00:27.9,None,1,C,200m,None,None,Sprint,...,2022,200m,13.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000
21830,21830,GOH EN YA ELEANA,00:26.2,None,1,A,200m,None,None,Sprint,...,2022,200m,18.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000
21831,21831,TONG YAN YEE,01:02.4,None,1,A,400m,None,None,Sprint,...,2022,400m,18.5,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200
21832,21832,LAAVINIA D/O JAIGANTH,01:00.3,None,1,B,400m,None,None,Sprint,...,2022,400m,16,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200


In [102]:
# Fill in '-' with np.NaN

relay_df['RESULT'] = relay_df['RESULT'].replace(regex=r'–', value=np.NaN)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/3629637655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relay_df['RESULT'] = relay_df['RESULT'].replace(regex=r'–', value=np.NaN)


In [103]:
# Reset index for index iteration below

relay_df=relay_df.reset_index(drop=True)

relay_df['index_x']=relay_df.index

relay_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,DATE,MAPPED_EVENT,MAPPED_AGE,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc
0,0,CHUA CHEE HON,00:20.5,None,4.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
1,1,O'NEAL TERENCE,00:33.3,None,5.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
2,2,MARCUS ANG GEE HONG,00:15.7,None,2.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
3,3,DHYAN JEEVANTH,00:17.4,None,3.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
4,4,MOHAMMAD ZULKHAIRI LUTFI BIN MOHD AZAM,00:13.9,None,1.0,A,Para 100m,None,None,Sprint,...,2023,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,9008,ELIZABETH LEE SHYIN,00:27.9,None,1,C,200m,None,None,Sprint,...,2022,200m,13.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000
9009,9009,GOH EN YA ELEANA,00:26.2,None,1,A,200m,None,None,Sprint,...,2022,200m,18.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000
9010,9010,TONG YAN YEE,01:02.4,None,1,A,400m,None,None,Sprint,...,2022,400m,18.5,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200
9011,9011,LAAVINIA D/O JAIGANTH,01:00.3,None,1,B,400m,None,None,Sprint,...,2022,400m,16,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200


In [104]:
#df = df.drop(['index', 'index_x'], axis=1)

In [105]:
relay_df.to_csv('df_relay.csv', sep=',', encoding='utf-8-sig', index=False)


In [106]:
# Convert results into seconds format

for i in range(len(relay_df)):
    
        
    rowIndex = relay_df.index[i]
    
        
    input_string=relay_df.iloc[rowIndex,14]    
    
    metric=relay_df.iloc[rowIndex,2]
        
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric==' DNS' or metric=='NH':
        continue
    
    print(i, input_string, metric)
        
    out = convert_time(i, input_string, metric)
         
    relay_df.loc[rowIndex, 'RESULT_CONV'] = out

0 100m 00:20.5
1 100m 00:33.3
2 100m 00:15.7
3 100m 00:17.4
4 100m 00:13.9
5 100m 00:14.7
6 100m 00:17.2
7 100m 00:14.7
8 100m 00:15.7
9 100m 00:14.0
10 100m 00:14.9
11 100m 00:14.2
12 100m 00:14.6
13 100m 00:13.7
14 100m 00:18.4
15 100m 00:15.8
16 100m 00:21.6
17 100m 00:15.2
18 100m 00:23.4
19 100m 00:22.3
20 100m 00:16.8
21 100m 00:28.7
22 100m 00:11.9
23 100m 00:11.1
24 100m 00:12.7
25 100m 00:12.9
26 100m 00:12.2
27 100m 00:12.0
28 100m 00:12.5
29 100m 00:13.1
30 100m 00:12.9
31 100m 00:11.7
32 100m 00:12.0
33 100m 00:12.4
34 100m 00:11.6
35 100m 00:11.8
36 100m 00:12.2
37 100m 00:11.6
38 100m 00:13.2
39 100m 00:12.5
40 100m 00:12.4
41 100m 00:13.9
42 100m 00:11.6
43 100m 00:12.1
44 100m 00:13.9
45 100m 00:12.7
46 100m 00:12.8
47 100m 00:12.0
48 100m 00:12.9
49 100m 00:12.2
50 100m 00:13.4
51 100m 00:13.0
52 100m 00:11.3
53 100m 00:11.9
54 100m 00:12.6
55 100m 00:11.6
56 100m 00:11.8
57 100m 00:12.9
58 100m 00:13.2
59 100m 00:13.9
60 100m 00:11.4
61 100m 00:12.5
62 100m 00:12.3
63

1874 400m 01:02.2
1875 400m 00:56.0
1876 400m 00:57.4
1877 400m 01:02.3
1878 400m 00:57.1
1879 400m 01:07.8
1880 400m 00:56.3
1881 400m 01:07.2
1882 400m 01:03.9
1883 400m 01:01.7
1884 400m 01:16.8
1885 400m 01:04.6
1886 400m 01:04.9
1887 400m 01:31.3
1888 400m 01:04.7
1889 400m 00:54.8
1890 400m 01:00.1
1891 400m 00:55.5
1892 400m 00:54.2
1893 400m 01:08.0
1894 400m 00:59.7
1895 400m 00:54.3
1896 400m 01:01.5
1897 400m 01:00.4
1898 400m 01:14.0
1899 400m 01:05.4
1900 400m 01:17.6
1901 400m 01:20.5
1902 400m 01:02.0
1903 400m 00:58.1
1904 400m 01:10.0
1905 400m 01:00.4
1906 400m 01:04.8
1907 400m 01:00.8
1908 400m 01:03.2
1909 400m 01:01.3
1910 400m 01:05.9
1911 400m 01:10.8
1912 400m 01:10.2
1913 400m 01:00.5
1914 400m 01:12.8
1915 400m 01:15.7
1916 400m 01:07.8
1917 400m 00:58.2
1918 400m 00:57.1
1919 400m 01:04.7
1920 400m 01:00.6
1921 400m 00:58.7
1922 400m 01:00.8
1923 400m 01:04.5
1924 400m 00:55.8
1925 400m 00:57.9
1926 400m 01:03.6
1927 400m 00:57.1
1928 400m 01:01.2
1929 400m 

3710 400m 1:14.69
3711 400m 59.15
3712 400m 1:08.99
3713 400m 58.00
3714 100m 14.47
3715 100m 12.62
3716 100m 13.09
3717 100m 13.85
3718 100m 14.06
3719 400m 58.60
3720 100m 14.84
3721 100m 14.89
3722 100m 14.08
3723 100m 12.80
3724 100m 15.08
3725 100m 15.52
3726 400m 57.88
3727 400m 1:11.08
3728 400m 1:00.87
3729 400m 1:05.35
3730 400m 1:12.91
3731 400m 1:12.67
3732 400m 57.01
3733 100m 12.33
3734 100m 12.12
3735 100m 13.05
3736 200m 25.94
3737 200m 25.94
3738 200m 25.94
3739 200m 25.91
3740 200m 25.91
3741 200m 25.91
3742 200m 27.11
3743 200m 27.11
3744 200m 27.11
3745 200m 25.36
3746 200m 25.36
3747 200m 25.36
3748 200m 25.08
3749 200m 25.08
3750 200m 25.08
3751 200m 27.64
3752 200m 27.64
3753 200m 27.64
3754 200m 24.52
3755 200m 24.52
3756 200m 24.52
3757 100m 11.82
3758 100m 12.98
3759 100m 13.13
3760 400m 57.10
3761 400m 1:00.17
3762 400m 1:01.45
3763 400m 1:04.51
3764 400m 1:04.22
3765 400m 55.14
3766 100m 13.06
3767 100m 13.86
3768 100m 14.12
3769 100m 13.28
3770 100m 14.84
37

5551 100m 00:16.0
5552 100m 00:15.4
5553 100m 00:14.8
5554 100m 00:15.4
5555 100m 00:15.5
5556 100m 00:18.8
5557 100m 00:15.1
5558 100m 00:15.3
5559 100m 00:16.4
5560 100m 00:14.1
5561 100m 00:15.0
5562 100m 00:18.0
5563 100m 00:15.1
5564 100m 00:16.7
5565 100m 00:19.4
5566 100m 00:15.0
5567 100m 00:17.9
5568 100m 00:15.7
5569 100m 00:12.8
5570 100m 00:14.5
5571 100m 00:18.2
5572 100m 00:15.7
5573 100m 00:14.4
5574 100m 00:15.6
5575 100m 00:15.3
5576 100m 00:16.1
5577 100m 00:15.4
5578 100m 00:18.1
5579 100m 00:17.3
5580 100m 00:14.3
5581 100m 00:19.0
5582 100m 00:17.2
5583 100m 00:15.5
5584 100m 00:13.4
5585 100m 00:18.0
5586 100m 00:15.0
5587 100m 00:14.8
5588 100m 00:18.2
5589 100m 00:17.8
5590 100m 00:16.4
5591 100m 00:15.9
5592 100m 00:16.3
5593 100m 00:14.7
5594 100m 00:15.0
5595 100m 00:19.0
5596 100m 00:15.2
5597 100m 00:15.2
5598 100m 00:14.9
5599 100m 00:14.6
5600 100m 00:15.3
5601 100m 00:14.4
5602 100m 00:17.1
5603 100m 00:15.8
5604 100m 00:15.3
5605 100m 00:15.2
5606 100m 

7272 100m 15.00
7273 100m 16.91
7274 100m 16.53
7275 200m 28.54
7276 200m 28.63
7277 200m 34.22
7278 100m 14.74
7279 100m 16.08
7280 100m 14.90
7281 100m 16.07
7282 100m 15.97
7283 100m 15.06
7284 100m 15.10
7285 100m 16.41
7286 100m 14.01
7287 100m 12.97
7288 100m 13.69
7289 100m 13.57
7290 100m 12.65
7291 100m 12.99
7292 100m 14.36
7293 100m 14.30
7294 100m 14.20
7295 100m 14.26
7296 100m 13.60
7297 100m 14.27
7298 100m 13.89
7299 100m 14.33
7300 100m 13.67
7301 100m 14.63
7302 100m 15.45
7303 100m 13.77
7304 100m 16.82
7305 100m 14.61
7306 100m 17.40
7307 100m 15.04
7308 100m 15.61
7309 100m 14.40
7310 100m 16.44
7311 100m 15.53
7312 100m 14.33
7313 100m 16.04
7314 100m 14.99
7315 100m 15.02
7316 100m 15.86
7317 100m 14.06
7318 100m 14.47
7319 100m 13.46
7320 100m 16.18
7321 100m 15.69
7322 100m 14.86
7323 100m 14.50
7324 100m 14.95
7325 100m 15.96
7326 100m 13.34
7327 100m 16.42
7328 100m 17.02
7329 100m 17.07
7330 100m 14.58
7331 100m 13.24
7332 100m 13.87
7333 100m 16.86
7334 100

In [107]:
relay_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,MAPPED_EVENT,MAPPED_AGE,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,CHUA CHEE HON,00:20.5,None,4.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,20.5
1,1,O'NEAL TERENCE,00:33.3,None,5.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,33.3
2,2,MARCUS ANG GEE HONG,00:15.7,None,2.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,15.7
3,3,DHYAN JEEVANTH,00:17.4,None,3.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,17.4
4,4,MOHAMMAD ZULKHAIRI LUTFI BIN MOHD AZAM,00:13.9,None,1.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,9008,ELIZABETH LEE SHYIN,00:27.9,None,1,C,200m,None,None,Sprint,...,200m,13.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000,27.9
9009,9009,GOH EN YA ELEANA,00:26.2,None,1,A,200m,None,None,Sprint,...,200m,18.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000,26.2
9010,9010,TONG YAN YEE,01:02.4,None,1,A,400m,None,None,Sprint,...,400m,18.5,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200,62.4
9011,9011,LAAVINIA D/O JAIGANTH,01:00.3,None,1,B,400m,None,None,Sprint,...,400m,16,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200,60.3


In [108]:
# Fill in any empty age fields

relay_df["AGE"].fillna(0, inplace=True)
relay_df['AGE'] = relay_df['AGE'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/4200550883.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  relay_df["AGE"].fillna(0, inplace=True)


In [32]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [33]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [34]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Mid')|(df['CATEGORY_EVENT']=='Sprint')|(df['CATEGORY_EVENT']=='Long')|(df['CATEGORY_EVENT']=='Hurdles')|(df['CATEGORY_EVENT']=='Walk')|(df['CATEGORY_EVENT']=='Relay')|(df['CATEGORY_EVENT']=='Marathon')|(df['CATEGORY_EVENT']=='Steeple')|(df['CATEGORY_EVENT']=='Pentathlon')|(df['CATEGORY_EVENT']=='Heptathlon')|(df['CATEGORY_EVENT']=='Triathlon'))&(df['RESULT_CONV'] <= df['5pc']) & (df['AGE']<40) & ((df['MAPPED_EVENT']!='Marathon')|(df['AGE']<60) & (df['MAPPED_EVENT']=='Marathon')))|(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['MAPPED_EVENT']!='Marathon')|(df['AGE']<60) & (df['MAPPED_EVENT']=='Marathon')))]

In [109]:
relay_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,MAPPED_EVENT,MAPPED_AGE,index_y,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV
0,0,CHUA CHEE HON,00:20.5,0.0,4.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,20.5
1,1,O'NEAL TERENCE,00:33.3,0.0,5.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,33.3
2,2,MARCUS ANG GEE HONG,00:15.7,0.0,2.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,15.7
3,3,DHYAN JEEVANTH,00:17.4,0.0,3.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,17.4
4,4,MOHAMMAD ZULKHAIRI LUTFI BIN MOHD AZAM,00:13.9,0.0,1.0,A,Para 100m,None,None,Sprint,...,100m,18.5,0.0,10.443,100m,10.443,10.65186,10.808505,10.96515,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,9008,ELIZABETH LEE SHYIN,00:27.9,0.0,1,C,200m,None,None,Sprint,...,200m,13.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000,27.9
9009,9009,GOH EN YA ELEANA,00:26.2,0.0,1,A,200m,None,None,Sprint,...,200m,18.5,22.0,23.6,200m,23.600,24.07200,24.426000,24.78000,26.2
9010,9010,TONG YAN YEE,01:02.4,0.0,1,A,400m,None,None,Sprint,...,400m,18.5,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200,62.4
9011,9011,LAAVINIA D/O JAIGANTH,01:00.3,0.0,1,B,400m,None,None,Sprint,...,400m,16,23.0,53.84,400m,53.840,54.91680,55.724400,56.53200,60.3


In [111]:
# Calculate performance deltas

mask = relay_df['CATEGORY_EVENT'].str.contains(r'Jump|Throw', na=True)
relay_df.loc[mask, 'Delta2'] = relay_df['RESULT_CONV']-relay_df['2pc']
relay_df.loc[mask, 'Delta35'] = relay_df['RESULT_CONV']-relay_df['35pc']
relay_df.loc[mask, 'Delta5'] = relay_df['RESULT_CONV']-relay_df['5pc']

relay_df.loc[~mask, 'Delta2'] =  relay_df['2pc'] - relay_df['RESULT_CONV']
relay_df.loc[~mask, 'Delta35'] = relay_df['35pc'] - relay_df['RESULT_CONV']
relay_df.loc[~mask, 'Delta5'] = relay_df['5pc'] - relay_df['RESULT_CONV']



#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
relay_df=relay_df.loc[relay_df['COMPETITION']!='SEA Games']

In [112]:
relay_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,BENCHMARK,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5
0,0,CHUA CHEE HON,00:20.5,0.0,4.0,A,Para 100m,None,None,Sprint,...,10.443,100m,10.443,10.65186,10.808505,10.96515,20.5,-9.84814,-9.691495,-9.53485
1,1,O'NEAL TERENCE,00:33.3,0.0,5.0,A,Para 100m,None,None,Sprint,...,10.443,100m,10.443,10.65186,10.808505,10.96515,33.3,-22.64814,-22.491495,-22.33485
2,2,MARCUS ANG GEE HONG,00:15.7,0.0,2.0,A,Para 100m,None,None,Sprint,...,10.443,100m,10.443,10.65186,10.808505,10.96515,15.7,-5.04814,-4.891495,-4.73485
3,3,DHYAN JEEVANTH,00:17.4,0.0,3.0,A,Para 100m,None,None,Sprint,...,10.443,100m,10.443,10.65186,10.808505,10.96515,17.4,-6.74814,-6.591495,-6.43485
4,4,MOHAMMAD ZULKHAIRI LUTFI BIN MOHD AZAM,00:13.9,0.0,1.0,A,Para 100m,None,None,Sprint,...,10.443,100m,10.443,10.65186,10.808505,10.96515,13.9,-3.24814,-3.091495,-2.93485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,9008,ELIZABETH LEE SHYIN,00:27.9,0.0,1,C,200m,None,None,Sprint,...,23.6,200m,23.600,24.07200,24.426000,24.78000,27.9,-3.82800,-3.474000,-3.12000
9009,9009,GOH EN YA ELEANA,00:26.2,0.0,1,A,200m,None,None,Sprint,...,23.6,200m,23.600,24.07200,24.426000,24.78000,26.2,-2.12800,-1.774000,-1.42000
9010,9010,TONG YAN YEE,01:02.4,0.0,1,A,400m,None,None,Sprint,...,53.84,400m,53.840,54.91680,55.724400,56.53200,62.4,-7.48320,-6.675600,-5.86800
9011,9011,LAAVINIA D/O JAIGANTH,01:00.3,0.0,1,B,400m,None,None,Sprint,...,53.84,400m,53.840,54.91680,55.724400,56.53200,60.3,-5.38320,-4.575600,-3.76800


In [113]:
# Correct some known names

relay_df['NAME'] = relay_df['NAME'].replace(regex=r'PRAHARSH, RYAN', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'Ryan, Praharsh', value='S/O SUBASH SOMAN, PRAHARSH RYAN')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'TAN, ELIZABETH ANN SHEE R', value='TAN, ELIZABETH-ANN')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'Tan, Elizabeth Ann', value='TAN, ELIZABETH-ANN')

relay_df['NAME'] = relay_df['NAME'].replace(regex=r'LOUIS, MARC BRIAN', value='Louis, Marc Brian')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'Louis, Marc', value='Louis, Marc Brian')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'TAN JUN JIE', value='Tan Jun Jie')

relay_df['NAME'] = relay_df['NAME'].replace(regex=r'SNG, MICHELLE', value='Sng, Michelle')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'SNG, SUAT LI, MICHELLE', value='Sng, Michelle')

relay_df['NAME'] = relay_df['NAME'].replace(regex=r'MUN, IVAN', value='Mun, Ivan')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'LOW, JUN YU', value='Low, Jun Yu')

relay_df['NAME'] = relay_df['NAME'].replace(regex=r'ANG, CHEN XIANG', value='Ang, Chen Xiang')
relay_df['NAME'] = relay_df['NAME'].replace(regex=r'LIM, OLIVER', value='Lim, Oliver')



In [114]:
# Create a perf_scalar variable to measure proximity to 5% window

relay_df['PERF_SCALAR']=relay_df['Delta5']/relay_df['Metric']*100

In [115]:
relay_df.to_csv('checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [41]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


In [116]:
spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [117]:
spex_list

,V1,V2,V3,V4,Level
0,Goh Chui Ling,"Goh, Chui Ling",NaN,NaN,E3
1,Michelle Sng,"Sng, Michelle",NaN,NaN,E3
2,Quek Jun Jie Calvin,"Quek, Jun Jie Calvin",NaN,NaN,E3
3,"SOH RUI YONG, GUILLAUME","Soh Rui Yong, Guillaume",NaN,NaN,E3
4,Aaron Justin tan wen jie,"Tan Wen Jie, Aaron Justin",NaN,NaN,E3P
5,DANIEL LEOW SOON YEE,"Yee, Daniel Leow Soon",NaN,NaN,E3P
6,Joshua Chua,"Chua, Joshua",NaN,NaN,E3P
7,NG ZHI RONG RYAN RAPHAEL,"Ng Zhi Rong, Ryan Raphael","Raphael, Ryan",NaN,E3P
8,Wenli Rachel,"Wenli, Rachel",NaN,NaN,E3P
9,Wong YaoHan Melvin,"Wong YaoHan, Melvin",NaN,NaN,E3P


In [118]:
spex1 = spex_list['V1'].dropna().tolist()
spex2 = spex_list['V2'].dropna().tolist()
spex3 = spex_list['V3'].dropna().tolist()
spex4 = spex_list['V4'].dropna().tolist()


In [119]:
# Put all spex carded names into one single list 

spex_athletes=spex1+spex2+spex3+spex4

In [120]:
spex_athletes

['Goh Chui Ling',
 'Michelle Sng',
 'Quek Jun Jie Calvin',
 'SOH RUI YONG, GUILLAUME',
 'Aaron Justin tan wen jie',
 'DANIEL LEOW SOON YEE',
 'Joshua Chua',
 'NG ZHI RONG RYAN RAPHAEL',
 'Wenli Rachel',
 'Wong YaoHan Melvin',
 'XANDER HO ANN HENG',
 'Veronica Shanti Pereira',
 'Ang Chen Xiang',
 'KAMPTON KAM',
 'MARC BRIAN LOUIS',
 'Mark Lee Ren',
 'Reuben Rainer Lee Siong En',
 'ELIZABETH-ANN TAN SHEE RU',
 'Thiruben Thana Rajan',
 'Bhavna Gopikrishna',
 'Chloe Chee En-Ya',
 'Conrad Kangli Emery',
 'Harry Irfan Curran',
 'Huang weijun',
 'Jayden tan',
 'Koh Shun Yi Audrey',
 'LAAVINIA D/O JAIGANTH',
 'Lim Yee Chern Clara',
 'LOH DING RONG ANSON',
 'ONG YING TAT',
 'SONG EN XU REAGAN',
 'Subaraghav hari',
 'Teh Ying Shan',
 'YAN TEO',
 'ZHONG CHUHAN',
 'ESTHER TAY SHEE WEI',
 'Faith Ford',
 'Garrett Chua Je-An',
 'Lucas Fun',
 'Goh, Chui Ling',
 'Sng, Michelle',
 'Quek, Jun Jie Calvin',
 'Soh Rui Yong, Guillaume',
 'Tan Wen Jie, Aaron Justin',
 'Yee, Daniel Leow Soon',
 'Chua, Joshua',

In [121]:
# Convert names to lower case for easy compare

spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [122]:
spex_athletes_casefold

['goh chui ling',
 'michelle sng',
 'quek jun jie calvin',
 'soh rui yong, guillaume',
 'aaron justin tan wen jie',
 'daniel leow soon yee',
 'joshua chua',
 'ng zhi rong ryan raphael',
 'wenli rachel',
 'wong yaohan melvin',
 'xander ho ann heng',
 'veronica shanti pereira',
 'ang chen xiang',
 'kampton kam',
 'marc brian louis',
 'mark lee ren',
 'reuben rainer lee siong en',
 'elizabeth-ann tan shee ru',
 'thiruben thana rajan',
 'bhavna gopikrishna',
 'chloe chee en-ya',
 'conrad kangli emery',
 'harry irfan curran',
 'huang weijun',
 'jayden tan',
 'koh shun yi audrey',
 'laavinia d/o jaiganth',
 'lim yee chern clara',
 'loh ding rong anson',
 'ong ying tat',
 'song en xu reagan',
 'subaraghav hari',
 'teh ying shan',
 'yan teo',
 'zhong chuhan',
 'esther tay shee wei',
 'faith ford',
 'garrett chua je-an',
 'lucas fun',
 'goh, chui ling',
 'sng, michelle',
 'quek, jun jie calvin',
 'soh rui yong, guillaume',
 'tan wen jie, aaron justin',
 'yee, daniel leow soon',
 'chua, joshua',

In [123]:
# If athlete appears in more than one event, choose the best performing one

top_performers=relay_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [124]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
6811,6811,"yeo, janice",1:16.22,18.0,14,U20,Women 400 Meter Dash U20,2006-08-20,None,Sprint,...,400m,53.840,54.91680,55.724400,56.53200,76.22,-21.30320,-20.495600,-19.68800,-36.567608
7267,7267,"yeo, jalea",14.69,13.0,18,U15,Women 100 Meter Dash U15,2011-01-28,None,Sprint,...,100m,11.750,11.98500,12.161250,12.33750,14.69,-2.70500,-2.528750,-2.35250,-20.021277
6945,6945,"toh, chloe",13.94,15.0,10,U18,Women 100 Meter Dash U18,2009-01-13,None,Sprint,...,100m,11.750,11.98500,12.161250,12.33750,13.94,-1.95500,-1.778750,-1.60250,-13.638298
2013,2013,"teo, yan",11.53,18.0,3,A Div,Men 100 Meter Dash A Div,2005-06-18,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,11.53,-0.87814,-0.721495,-0.56485,-5.408886
2120,2120,"tay hong yan, jeryl",13.66,13.0,48,U15,Men 100 Meter Dash U15,2011-02-15,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,13.66,-3.00814,-2.851495,-2.69485,-25.805324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,921,AARON LUCAS WHALL,00:26.9,0.0,28.0,C,200m,None,None,Sprint,...,200m,21.020,21.44040,21.755700,22.07100,26.90,-5.45960,-5.144300,-4.82900,-22.973359
4569,4569,AARON KEITH IMMANUEL,00:24.1,0.0,8,B,200m,None,None,Sprint,...,200m,21.020,21.44040,21.755700,22.07100,24.10,-2.65960,-2.344300,-2.02900,-9.652712
5585,5585,A A Ayu Teja Gayatri,00:18.0,0.0,86.0,B,100m,None,None,Sprint,...,100m,11.750,11.98500,12.161250,12.33750,18.00,-6.01500,-5.838750,-5.66250,-48.191489
7686,7686,"., Kaarthika",1:59.53,27.0,9,Women Junior,Women 400 Meter Dash Women Junior,1996-06-13,None,Sprint,...,400m,53.840,54.91680,55.724400,56.53200,119.53,-64.61320,-63.805600,-62.99800,-117.009658


In [125]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)


In [126]:
# Filter out SPEX carded athletes from list

spexed_list = top_performers.loc[~relay_df['NAME'].str.casefold().isin(spex_athletes_casefold)]  # ~ means NOT IN. DROP spex carded athletes

In [127]:
# Filter 100m, 200m and 400m, age between 20 and 35

spexed_list['MAPPED_AGE'] = spexed_list['MAPPED_AGE'].astype(float)

ranked_df = spexed_list.loc[(spexed_list['MAPPED_AGE'] >= 20) & (spexed_list['MAPPED_AGE']<=35) & ((spexed_list['DATE']=='2023'))]
#ranked_df = spexed_list.loc[(spexed_list['MAPPED_AGE'] >= 18) & (spexed_list['MAPPED_AGE']<=19) & ((spexed_list['MAPPED_EVENT']=='100m')|(spexed_list['MAPPED_EVENT']=='200m')|(spexed_list['MAPPED_EVENT']=='400m'))]

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/1797569186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spexed_list['MAPPED_AGE'] = spexed_list['MAPPED_AGE'].astype(float)


In [128]:
ranked_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
3609,3609,"Zhen Yong, Chang",1:05.81,25.0,11,Men Junior,Men 400 Meter Dash Men Junior,1998-09-22,None,Sprint,...,400m,46.630,47.56260,48.262050,48.96150,65.81,-18.24740,-17.547950,-16.84850,-36.132318
3270,3270,"Yeo, Jeremy, Zhi Hui",16.84,20.0,2,Para,Men 100 Meter Dash Para,2002-11-14,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,16.84,-6.18814,-6.031495,-5.87485,-56.256344
2001,2001,"Yeo, Jeremy",17.74,21.0,1,Para,Men 100 Meter Dash Para,2002-11-14,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,17.74,-7.08814,-6.931495,-6.77485,-64.874557
7731,7731,"Wong, Jannah",14.49,27.0,1,Women Junior,Women 100 Meter Dash Women Junior,1996-08-09,None,Sprint,...,100m,11.750,11.98500,12.161250,12.33750,14.49,-2.50500,-2.328750,-2.15250,-18.319149
3617,3617,"Wan Hong, Chong",13.15,24.0,11,Men Junior,Men 100 Meter Dash Men Junior,1999-12-23,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,13.15,-2.49814,-2.341495,-2.18485,-20.921670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7683,7683,"Aw, Vanessa",1:13.92,28.0,1,Women Junior,Women 400 Meter Dash Women Junior,1995-09-11,None,Sprint,...,400m,53.840,54.91680,55.724400,56.53200,73.92,-19.00320,-18.195600,-17.38800,-32.295691
3403,3403,"Ariiq, Muhammad",13.45,22.0,14,Men Junior,Men 100 Meter Dash Men Junior,2001-09-08,None,Sprint,...,100m,10.443,10.65186,10.808505,10.96515,13.45,-2.79814,-2.641495,-2.48485,-23.794408
7751,7751,"Ang, Shermaine",17.72,23.0,9,Women Junior,Women 100 Meter Dash Women Junior,2000-01-14,None,Sprint,...,100m,11.750,11.98500,12.161250,12.33750,17.72,-5.73500,-5.558750,-5.38250,-45.808511
3509,3509,"Alexander Greaves, James",27.26,21.0,10,Men Junior,Men 200 Meter Dash Men Junior,2002-02-17,None,Sprint,...,200m,21.020,21.44040,21.755700,22.07100,27.26,-5.81960,-5.504300,-5.18900,-24.686013


In [129]:
# Rank and sort results

ranked_df.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
ranked_df['overall_rank'] = 1
ranked_df['overall_rank'] = ranked_df.groupby(['GENDER', 'MAPPED_EVENT'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/4071210366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranked_df.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/4071210366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranked_df['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_45688/4071210366.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [130]:
ranked_df

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
7731,7731,"Wong, Jannah",14.49,27.0,1,Women Junior,Women 100 Meter Dash Women Junior,1996-08-09,None,Sprint,...,11.75,11.9850,12.16125,12.3375,14.49,-2.5050,-2.32875,-2.1525,-18.319149,1
7654,7654,"Hui Ting, Liow",15.59,29.0,2,Women Junior,Women 100 Meter Dash Women Junior,1994-11-04,None,Sprint,...,11.75,11.9850,12.16125,12.3375,15.59,-3.6050,-3.42875,-3.2525,-27.680851,2
7685,7685,"Chua, Joelle",15.72,25.0,2,Women Junior,Women 100 Meter Dash Women Junior,1998-04-28,None,Sprint,...,11.75,11.9850,12.16125,12.3375,15.72,-3.7350,-3.55875,-3.3825,-28.787234,3
7671,7671,"Mualim, Nurul Natasha",16.03,25.0,4,Women Junior,Women 100 Meter Dash Women Junior,1998-05-24,None,Sprint,...,11.75,11.9850,12.16125,12.3375,16.03,-4.0450,-3.86875,-3.6925,-31.425532,4
7752,7752,"Vasudevan, Pushpaja",16.57,23.0,6,Women Junior,Women 100 Meter Dash Women Junior,2000-11-29,None,Sprint,...,11.75,11.9850,12.16125,12.3375,16.57,-4.5850,-4.40875,-4.2325,-36.021277,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,3505,"BIN FARED, AQMAL FIQRI",1:13.95,21.0,17,Men Junior,Men 400 Meter Dash Men Junior,2002-01-18,None,Sprint,...,46.63,47.5626,48.26205,48.9615,73.95,-26.3874,-25.68795,-24.9885,-53.588891,15
3575,3575,"Bin Md Akip, Muhammad Hilal",1:16.01,23.0,18,Men Junior,Men 400 Meter Dash Men Junior,2000-03-09,None,Sprint,...,46.63,47.5626,48.26205,48.9615,76.01,-28.4474,-27.74795,-27.0485,-58.006648,16
3390,3390,"Lim, Berton",1:16.62,29.0,19,Men Junior,Men 400 Meter Dash Men Junior,1994-06-07,None,Sprint,...,46.63,47.5626,48.26205,48.9615,76.62,-29.0574,-28.35795,-27.6585,-59.314819,17
3520,3520,"Azlyman, Mus",1:19.80,23.0,20,Men Junior,Men 400 Meter Dash Men Junior,2000-02-23,None,Sprint,...,46.63,47.5626,48.26205,48.9615,79.80,-32.2374,-31.53795,-30.8385,-66.134463,18


In [131]:
# Filter top 20

top_20_20_to_35=ranked_df[ranked_df['overall_rank']<21]


In [132]:
top_20_20_to_35

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
7731,7731,"Wong, Jannah",14.49,27.0,1,Women Junior,Women 100 Meter Dash Women Junior,1996-08-09,None,Sprint,...,11.75,11.9850,12.16125,12.3375,14.49,-2.5050,-2.32875,-2.1525,-18.319149,1
7654,7654,"Hui Ting, Liow",15.59,29.0,2,Women Junior,Women 100 Meter Dash Women Junior,1994-11-04,None,Sprint,...,11.75,11.9850,12.16125,12.3375,15.59,-3.6050,-3.42875,-3.2525,-27.680851,2
7685,7685,"Chua, Joelle",15.72,25.0,2,Women Junior,Women 100 Meter Dash Women Junior,1998-04-28,None,Sprint,...,11.75,11.9850,12.16125,12.3375,15.72,-3.7350,-3.55875,-3.3825,-28.787234,3
7671,7671,"Mualim, Nurul Natasha",16.03,25.0,4,Women Junior,Women 100 Meter Dash Women Junior,1998-05-24,None,Sprint,...,11.75,11.9850,12.16125,12.3375,16.03,-4.0450,-3.86875,-3.6925,-31.425532,4
7752,7752,"Vasudevan, Pushpaja",16.57,23.0,6,Women Junior,Women 100 Meter Dash Women Junior,2000-11-29,None,Sprint,...,11.75,11.9850,12.16125,12.3375,16.57,-4.5850,-4.40875,-4.2325,-36.021277,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,3505,"BIN FARED, AQMAL FIQRI",1:13.95,21.0,17,Men Junior,Men 400 Meter Dash Men Junior,2002-01-18,None,Sprint,...,46.63,47.5626,48.26205,48.9615,73.95,-26.3874,-25.68795,-24.9885,-53.588891,15
3575,3575,"Bin Md Akip, Muhammad Hilal",1:16.01,23.0,18,Men Junior,Men 400 Meter Dash Men Junior,2000-03-09,None,Sprint,...,46.63,47.5626,48.26205,48.9615,76.01,-28.4474,-27.74795,-27.0485,-58.006648,16
3390,3390,"Lim, Berton",1:16.62,29.0,19,Men Junior,Men 400 Meter Dash Men Junior,1994-06-07,None,Sprint,...,46.63,47.5626,48.26205,48.9615,76.62,-29.0574,-28.35795,-27.6585,-59.314819,17
3520,3520,"Azlyman, Mus",1:19.80,23.0,20,Men Junior,Men 400 Meter Dash Men Junior,2000-02-23,None,Sprint,...,46.63,47.5626,48.26205,48.9615,79.80,-32.2374,-31.53795,-30.8385,-66.134463,18


In [133]:
top_20_20_to_35.to_csv('top_20_20_to_35.csv', sep=',', encoding='utf-8-sig', index=False)


In [2021]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [305]:
#spexed_list=spexed_list[(((spexed_list['MAPPED_EVENT']=='400m')|(spexed_list['MAPPED_EVENT']=='100m'))&(spexed_list['overall_rank']<7))|(~((spexed_list['MAPPED_EVENT']=='400m')|(spexed_list['MAPPED_EVENT']=='100m'))&(spexed_list['overall_rank']<4))]


In [150]:
#spexed_list.to_csv('ranked_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?